In [13]:
import networkx as nx
import matplotlib.pyplot as plt
from qrisp import *
import numpy as np
from sympy import Symbol


In [14]:
def cost_symp(tot_coeff,n_parts,n_sites,Phi_graph):
    """
    returns the sympy cost function and the dictionary for the variables
    """
    combinations=list(tot_coeff[1].keys())

    x = {str(r)+str(i): Symbol(f"x{r}{i}") for r in range(n_parts) for i in range(n_sites)}

    cost=sum([ sum([ tot_coeff[r][(i,j)]*x[str(r)+str(i)]*x[str(s)+str(j)] for (i,j) in combinations]) for r,s in Phi_graph.edges()])

    return cost,x


# The following 2 methods are used to move from the one hot encoding of the states
# to a binary one, in order to evaluate the cost function above.
# This can be done surely better with some qrisp feature and it will be changed.

def convert_to_binary(outcome_array):
    binary_string = ''
    for element in outcome_array:
        binary_string += ''.join('1' if i == element else '0' for i in range(N-1 + 1))
    return binary_string[::-1]


def translate_encoding(outdic):
    return {convert_to_binary(k):v  for k,v in outdic.items()}

In [15]:
# Cost coefficients from Table.3
cost_coeff={
    1: {(0,1): 1.64,
        (0,2): 1.05,
        (1,2): 0.59,
        },

    2: {(0,1): 5.56,
        (0,2): 3.54,
        (1,2): 1.98,
        },

     3: {(0,1): 8.06,
         (0,2): 5.14,
         (1,2): 2.88,
         },
    }
tot_coeff = {}

for k,v in cost_coeff.items():
  tot_coeff[k]={}
  for rs,c in v.items():
    i,j=rs[0],rs[1]
    tot_coeff[k][(i,j)]=c
    tot_coeff[k][(j,i)]=c

In [16]:
Phi = [(1,0),(2,0)]
N=3
G = nx.DiGraph()
G.add_edges_from(Phi)
M = G.number_of_nodes() # Number of parts

qubo,symb_dic=cost_symp(tot_coeff,M,N,G)
ord_symbs=list(symb_dic.values()) #ordering the symbols for the encoding
qubo

1.64*x00*x11 + 1.05*x00*x12 + 5.56*x00*x21 + 3.54*x00*x22 + 1.64*x01*x10 + 0.59*x01*x12 + 5.56*x01*x20 + 1.98*x01*x22 + 1.05*x02*x10 + 0.59*x02*x11 + 3.54*x02*x20 + 1.98*x02*x21

In [17]:

def QmatrixFromQUBO(qubo,vars):
    # No need for the main diagonal due to the qubo PBS cost function (unconstrained)
    coeffdict=qubo.as_coefficients_dict()

    num_vars=len(vars)
    Q_matrix = np.zeros((num_vars, num_vars), dtype=float)

    for i,k in enumerate(vars):
        for j,z in enumerate(vars):
          if i<j: #only upper triangular
            Q_matrix[i, j] = coeffdict[k*z]
    return Q_matrix

In [18]:
Q=QmatrixFromQUBO(qubo,ord_symbs)
Q

array([[0.  , 0.  , 0.  , 0.  , 1.64, 1.05, 0.  , 5.56, 3.54],
       [0.  , 0.  , 0.  , 1.64, 0.  , 0.59, 5.56, 0.  , 1.98],
       [0.  , 0.  , 0.  , 1.05, 0.59, 0.  , 3.54, 1.98, 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ]])

In [19]:

def array_create_QUBO_cost_operator(Q):
    #Below N is the number of sites of the PBS.

    #W.r.t. the other implementation we move from qv[i] to qv[i//N][i%N] since qarg will be a QuantumArray

    def array_QUBO_cost_operator(qv, gamma):

        gphase(-gamma/4*(np.sum(Q)+np.trace(Q)),qv[0][0])
        for i in range(len(Q)):
            rz(-gamma/2*(sum(Q[i])+sum(Q[:,i])), qv[i//N][i%N])
            for j in range(len(Q)):
                if i != j and Q[i][j] != 0:
                    rzz(gamma/2*Q[i][j], qv[i//N][i%N], qv[j//N][j%N])
    return array_QUBO_cost_operator

In [ ]:

from qrisp.quantum_backtracking import OHQInt

qtype = OHQInt(N)
q_array = QuantumArray(qtype = qtype, shape = (M))
gamma = Symbol("gamma")
array_create_QUBO_cost_operator(Q)(q_array,gamma)
print(q_array.qs)

In [ ]:

def array_create_QUBO_cl_cost_function():
    def array_cl_cost_function(counts):

        vars_ = ord_symbs
        b_dic=translate_encoding(counts)
        cost=0
        for k,v in b_dic.items():
          qval = qubo.subs({var:s for var, s in zip(vars_, k)})
          cost +=  qval * v

        return cost

    return array_cl_cost_function